<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
from sklearn import model_selection
from skimage.transform import resize
import skimage
from skimage import io
import tensorflow.contrib.layers as layers
from random import shuffle

In [2]:
NUM_ITER = 1000
NUM_ITER = 1
IMG_SHAPE = (339, 401) # (145,145)
NUM_CLASSES = 20
LEARNING_RATE = .001
BATCH_SIZE = 15

In [3]:
def train_test_split(df):
    train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)
    return train_df, val_df

#Split the data into a training set, and test set 
def accuracy(pred, actual):
    return np.mean(pred == actual)
# Calculate the accuracy percentage of the predicted values
def separate_features(df):
    feature_df = df.iloc[:,2:]
    labels_df = df.iloc[:,1]
    return feature_df, labels_df

In [4]:
training_data = pd.read_hdf('training_data.h5')

In [5]:
train, val = train_test_split(training_data)
# X_train, y_train = separate_features(train)
# X_train.reset_index().drop(columns=['index'])
# y_train.reset_index().drop(columns=['index'])
# y_train = y_train.astype('int')
# X_train_matrix = X_train.values
X_train_df = train['image']
y_train_df = train['label'].astype('int')
# X_train.head()

In [6]:
def preprocess(X):
    def preprocess_image(image):
        return resize(image / 255.0, IMG_SHAPE, mode='constant')
    images = []
    heights, widths = [], []
    for image in X:
#         heights.append(image.shape[0])
#         widths.append(image.shape[1])
        images.append(preprocess_image(image))
#     print(np.mean(heights), np.mean(widths))
    return np.array(images)
X_train = preprocess(X_train_df)
y_train = np.zeros((len(y_train_df), NUM_CLASSES))
y_train[np.arange(len(y_train_df)), y_train_df.values] = 1

In [7]:
def get_model():
    X_ph = tf.placeholder(tf.float32, shape=[None] + list(IMG_SHAPE) + [3])
    out = layers.convolution2d(X_ph, num_outputs=32, kernel_size=8, stride=4, activation_fn=tf.nn.relu)
    out = layers.convolution2d(out, num_outputs=64, kernel_size=4, stride=2, activation_fn=tf.nn.relu)
    out = layers.convolution2d(out, num_outputs=64, kernel_size=3, stride=1, activation_fn=tf.nn.relu)

    out = layers.flatten(out)
    out = layers.fully_connected(out, num_outputs=512,         activation_fn=tf.nn.relu)
    y_hat = layers.fully_connected(out, num_outputs=NUM_CLASSES, activation_fn=None)

    # Alexnet architecture (without batch norm)
#     out = layers.convolution2d(X_ph, num_outputs=96, kernel_size=11, stride=4, activation_fn=tf.nn.relu)
#     out = tf.layers.max_pooling2d(out, pool_size=3, strides=2)
#     out = layers.convolution2d(out, num_outputs=256, kernel_size=5, stride=1, activation_fn=tf.nn.relu)
#     out = tf.layers.max_pooling2d(out, pool_size=3, strides=2)
# #     out = layers.convolution2d(out, num_outputs=384, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
# #     out = layers.convolution2d(out, num_outputs=384, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
# #     out = layers.convolution2d(out, num_outputs=256, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
# #     out = tf.layers.max_pooling2d(out, pool_size=3, strides=2)
#     out = layers.flatten(out)
#     out = layers.fully_connected(out, num_outputs=256,         activation_fn=tf.nn.relu)
#     y_hat = layers.fully_connected(out, num_outputs=NUM_CLASSES,         activation_fn=tf.nn.relu)
    return X_ph, y_hat

In [8]:
tf.reset_default_graph()
sess = tf.Session()

indices = np.arange(len(X_train))
shuffle(indices)
print(X_train.shape)
X_train = X_train[indices]
y_train = y_train[indices]
# print(y_train[0:100])

X_ph, y_hat = get_model()
# print(X_ph.get_shape(), X_train.shape)
# print(X_train.head(20))
y_ph = tf.placeholder(tf.float32, shape=[None, NUM_CLASSES])
# loss = tf.losses.mean_squared_error(y_hat, y_ph)
loss = tf.reduce_mean(tf.square(y_hat - y_ph))
train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
sess.run(tf.global_variables_initializer())

for iteration in range(NUM_ITER):
    if iteration % 100 == 0:
        indices = np.arange(len(X_train))
        shuffle(indices)
        indices = indices[:200]
        print(indices)
        pred = sess.run(y_hat, feed_dict={X_ph: X_train[indices]})
#         print('pred', pred)
#         print('y', y_train[indices])
        curr_loss = sess.run(loss, feed_dict={X_ph: X_train[indices], y_ph: y_train[indices]})
        print('Loss: {}'.format(curr_loss))
        
    i = iteration % len(X_train)
    X_batch = X_train[i:i+BATCH_SIZE]
    y_batch = y_train[i:i+BATCH_SIZE]
    sess.run(train_step, feed_dict={X_ph: X_batch, y_ph: y_batch})

(1200, 339, 401, 3)
[ 648 1161  947 1069  290  101  251  244  259 1106  734  912 1121  505  625
   31  853 1089  606  459  387  851  885  915  143 1118  754   23  223  246
  345  415  839  563  819  502  340  419  768 1050  110  821  441  247  712
  895  405 1176   39  994   78  260  249   11  726  838   69   92  730  694
  159   48  766  199 1000   24  615 1056  745 1024    0   95  525  400   89
  781  208 1115  286 1097  446  652   62   12   51  631  113 1107  556  250
  416  140  919  102  571  782  762 1103  298 1187  136 1136 1027 1135  258
  587  538  528  149  725  417  356  728  285  964   82  988  218  299 1199
  641 1094  578  707  883  225 1010  804  767 1138  865  618  976   36 1137
  596  626  948 1148  237   38  353  253  991  452   65 1032  941  331 1029
  604  121 1046  545  396  507  719  153  814  407  756  910  901  479  568
  491  582  518   94  343  936  313  699  448  182  637  939 1143  501  377
 1018  670  267 1151 1035  922  229  372 1066  139 1001  393  363 10

In [9]:
X_val_df = val['image']
X_val = preprocess(X_val_df)
y_predicted = sess.run(y_hat, feed_dict={X_ph: X_val})
y_predicted = np.argmax(y_predicted, axis=1)
accuracy(y_predicted, val['label'].astype('int'))

0.039867109634551492